# Letzter Stand
- Meeting mit Informatikperson: 7. September 11:00
- wrote script to filter out syncretic forms. Als script in `preprocessing.py` und hier im notebook. 

Todo:
- Fact-check output of preprocessing script. 
- Check the re-shaping in der k means funktion. 
    - Warum muss das gereshaped werden? Warum muss der Input 2d sein?
    - ist der Input falsch wenn ich den noch umformen muss, oder hat anastasia das auch gemacht?
- Visualisierungs Funktion für die cluster die entstehen
- Das ganze in eine script

In [1]:
from sklearn.cluster import KMeans
import fasttext as ft
import numpy as np

- die scikit learn documentation empfiehlt PCA vor k means laufen zu lassen und wird machen das Gegenteil??
- k means sucht nach cluster with equal variance, ist das bei unseren clulstern gegeben?

In [2]:
# Load data.
forms_full = [
    {"nomn"},
    {"gent"},
    {"datv"},
    {"accs"},
    {"ablt"},
    {"loct"},
    {"gen2"},
    {"plur", "nomn"},
    {"plur", "gent"},
    {"plur", "datv"},
    {"plur", "accs"},
    {"plur", "ablt"},
    {"plur", "loct"},
    {"plur", "gen2"},
]

# For each noun: Lemma, tags (gender, animacy), something, all 14 cases (see above).
list_of_all_forms = np.loadtxt(
    "../../data/list_of_noun_forms_full.csv", dtype=object, delimiter=","
)

# Binary coding of synchretisms. A form at index i,j is not syncretic if matrix_of_syncr[i][j]==2**j.
matrix_of_syncr = np.loadtxt(
    "../../data/matrix_of_syncr.csv", dtype=float, delimiter=","
).astype(int)

# List of cases without lemma and tags.
list_forms = list_of_all_forms[:, 4:]

# Load fasttext model.
ft = ft.load_model("../../data/final-model.bin")

# Get all the vectors in the array.
# Gets the vector for all word lemmas from list_forms.
vectors_all_nouns = [[ft.get_word_vector(x) for x in line] for line in list_forms]

In [3]:
# Get all vectors that are non synchretic from fasttext.
# INCORRECT...
nonsynchr_nounvectors = []
for index, forms in enumerate(list_forms):
    for form in forms:
        for j in range(0, 14):
            if matrix_of_syncr[index][j] == 2**j:  # Check if word is synchretic.
                nonsynchr_nounvectors.append(ft.get_word_vector(form))
print(len(nonsynchr_nounvectors), len(list_forms))  # 840 11515

961618 11515


In [54]:
np.save("../../data/nonsynchr_casevectors.npy", nonsynchr_casevectors)

# Removing synchronous forms

`list_forms` and `matrix of synchr.` have the same length so presumably the same structure.
> 11515 lines and 14 dims per line 

```python
matrix_of_syncr[0]
array([   1,   74,    4,   74,   16,   32,   74,  128, 9472,  512, 9472,
       2048, 4096, 9472])
```

```python
list_forms[0]
array(['человек', 'человека', 'человеку', 'человека', 'человеком',
       'человеке', 'человека', 'люди', 'людей', 'людям', 'людей',
       'людьми', 'людях', 'людей'], dtype=object)
```

In [49]:
# Get case vectors for all non-synchretic cases.
nonsynchr_casevectors = []
for form in range(0, len(list_forms)):
    for case in range(0, 14):
        if (
            matrix_of_syncr[form][case] == 2**case
        ):  # Get word vector if case is not synchretic.
            vector = ft.get_word_vector(list_forms[form][case])
            nonsynchr_casevectors.append(vector)
print(len(nonsynchr_casevectors), len(list_forms))  # 68687 11515

### Official tries

In [14]:
def cluster_kmeans(n_clusters, input):
    """Cluster the input into n_clusters using kmeans.

    Parameters
    ----------
    n_clusters: int
        Number of clusters to cluster the input into.
    input: array-like
        Input to cluster.

    Returns
    -------
    labels: array-like
        Cluster labels for each element in the input.
    centers: array-like
        Cluster centers.
    """
    x = np.ascontiguousarray(
        input
    )  # Kmeans function copies input if input is not c-contigous.
    nsamples, nx, ny = x.shape
    x = x.reshape((nsamples, nx * ny))

    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(x)
    return kmeans.labels_, kmeans.cluster_centers_

In [ ]:
labels, centers = cluster_kmeans(
    8, vectors_all_nouns
)  # Whats a good number of clusters?
labels, centers